In [22]:
# importanto bibliotecas
import pandas as pd
import numpy as np
import psycopg2

#pacotes gráficos
import seaborn as sns
import matplotlib.pyplot as plt

#pacote de validação de email
from validate_email  import validate_email

# biblioteca de statística
import statistics
from sklearn.model_selection import StratifiedShuffleSplit
import scipy.stats as stats

#biblioteca de sistema operacional
import os

In [23]:
# conexão redshift 
connection = psycopg2.connect(dbname="dw_amorsaude",

                  user="lucasmateus",

                  password="df93aS@aK0sd",

                  host ="amorsaude-cluster-redshift.cmc4ev2oyczd.us-east-1.redshift.amazonaws.com",

                  port = 5439)

In [24]:
# definição queries

query_total =  """with agendamentos as (
	select max(ag.datadoatendimento) dt_atendimento, ag.id_regional, ag.descricao_unidades_regioes regional, ag.id_unidade, 
	ag.nome_fantasia, ag.id_especialidade, ag.nome_especialidade, ag.nome_procedimento, ag.id_paciente, ag.nome_paciente, 
	sp.celular,sp.email 
	from tb_consolidacao_agendamentos_hist ag
	join stg_pacientes sp on sp.id = ag.id_paciente
	where ag.datadoatendimento >= dateadd(day, -60, current_date)
	and sp.celular is not null
	and length(sp.celular) > 0
	and sp.email is not null
	and length(sp.email) > 0
	and ag.id_regional in (3, 16, 9, 10, 8, 6, 13, 7)
	--and ag.id_regional = 
	group by ag.id_regional, ag.descricao_unidades_regioes, ag.id_unidade, ag.nome_fantasia, ag.id_especialidade, 
	ag.nome_especialidade, ag.nome_procedimento, ag.id_paciente, ag.nome_paciente, sp.celular, sp.email),
qtd_atendimentos as (
select ag.id_unidade, count(ag.datadoatendimento) qtd_atendimentos
from tb_consolidacao_agendamentos_hist ag
where ag.datadoatendimento >= dateadd(day, -60, current_date)
and ag.status_id in (33, 207, 202, 2, 200, 203, 5, 204, 201, 205, 4, 206, 3)
group by ag.id_unidade),
sexo_idade as (
	select pa.id as id_paciente, datediff(year, pa.nascimento, getdate()) as idade, pa.sexo, sx.nomesexo
	from stg_pacientes pa
	left join stg_sexo sx on sx.id = pa.sexo
	where pa.nascimento is not null 
	and pa.sexo is not null
	and pa.sexo <> 0
	and datediff(year, pa.nascimento, getdate()) between 0 and 100)
select ag.*, si.idade, si.sexo, si.nomesexo from agendamentos ag
left join qtd_atendimentos att on ag.id_unidade = att.id_unidade
join sexo_idade si on si.id_paciente = ag.id_paciente
where att.qtd_atendimentos >= 2000"""

In [25]:
# setando o dataframe completo

df = pd.read_sql_query(query_total,con=connection)

/tmp/ipykernel_676/2585826873.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_total,con=connection)


In [26]:
df

,dt_atendimento,id_regional,regional,id_unidade,nome_fantasia,id_especialidade,nome_especialidade,nome_procedimento,id_paciente,nome_paciente,celular,email,idade,sexo,nomesexo
0,2023-06-15,16,SP CAV,19438,Clínica Taubaté (Taubaté),128.0,Ginecologia e Obstetrícia,Consulta Ginecologia,59865209,Pamela Dos Santos Tupinamba Castilho,(12) 99189-3999,tupi@gmail.com,29,2.0,Feminino
1,2023-07-04,8,ES,19458,AmorSaúde Cariacica,129.0,Ortopedia e Traumatologia,Consulta Ortopedia traumatologia,62804265,Helena Noemia Zahn,(27) 99751-0594,helenazahn@gmail.com,63,2.0,Feminino
2,2023-05-31,3,SP Interior,19843,AmorSaúde Peruíbe,129.0,Ortopedia e Traumatologia,Consulta Ortopedia traumatologia,65800067,SHEILA DE OLIVEIRA HORVATE,(13) 99687-1008,sheilahorvate906@gmail.com,32,2.0,Feminino
3,2023-06-22,7,CO,19320,AmorSaúde Ceilândia Norte,188.0,Psicologia,Sessão Psicologia,63364153,Maria Eduarda Oliveira Belchior,(61) 99155-4502,raquelvuar6@gmail.com,17,2.0,Feminino
4,2023-05-29,16,SP CAV,19359,AmorSaúde SP Jabaquara,188.0,Psicologia,Retorno Psicologia,64393064,Anderson Ferreira Crispim,(11) 96286-5263,cintialainne@ymail.com,48,1.0,Masculino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090487,2023-07-06,16,SP CAV,19339,AmorSaúde Guarulhos Parque São Luiz,271.0,Ginecologia,Consulta Ginecologia,13733622,VICTORIA HELEN DE LIMA,(11) 94989-0268,victoriadelima@outlook.com.br,26,2.0,Feminino
1090488,2023-06-01,9,MG,19314,AmorSaúde Betim,141.0,Psiquiatria,Sessão Psiquiatria,18390963,Valeria Barbosa Santos Galdino,(31) 98538-3103,Clientesmorsaude@outlook.com,51,2.0,Feminino
1090489,2023-07-24,3,SP Interior,19400,AmorSaúde Piracicaba,128.0,Ginecologia e Obstetrícia,Consulta Ginecologia,11450354,Vanderleia Mota Neves Papa,(19) 99890-4970,Vanderleia365@gmail.com,57,2.0,Feminino
1090490,2023-05-19,13,RJ,19563,AmorSaúde RJ Campo Grande,89.0,Clinica Médica,Consulta Clinica Médica,58815041,Beatriz Carvalho Silva Venâncio,(21) 96515-6420,BEATRZ@YAHOO.COM,25,2.0,Feminino


In [27]:
# validando os registros de email

valid_emails = []

for index, row in df.iterrows():
    email = row['email']
    if validate_email(email):
        valid_emails.append(index)

In [28]:
valid_df = df.loc[valid_emails]


In [38]:
invalid_emails = []

for index, row in df.iterrows():
    email = row['email']
    if not validate_email(email):
        invalid_emails.append(index)

invalid_df = df.loc[invalid_emails]

In [39]:
# exportando df com emails invalidos para feegow arrumar
emails_invalidos = 'base_emails_invalidos.xlsx'

invalid_df.to_excel(emails_invalidos, index=False)

In [29]:
# criando lista com grupos de idade
age_groups = ['Abaixo 18', '18-25', '25-35', '35-50', '50-65', 'Acima 65']

# criando coluna de grupo de idade
valid_df['grupo_idade'] = pd.cut(valid_df['idade'], bins=[0, 18, 25, 35, 50, 65, np.inf], labels=age_groups)

valid_df

,dt_atendimento,id_regional,regional,id_unidade,nome_fantasia,id_especialidade,nome_especialidade,nome_procedimento,id_paciente,nome_paciente,celular,email,idade,sexo,nomesexo,grupo_idade
0,2023-06-15,16,SP CAV,19438,Clínica Taubaté (Taubaté),128.0,Ginecologia e Obstetrícia,Consulta Ginecologia,59865209,Pamela Dos Santos Tupinamba Castilho,(12) 99189-3999,tupi@gmail.com,29,2.0,Feminino,25-35
1,2023-07-04,8,ES,19458,AmorSaúde Cariacica,129.0,Ortopedia e Traumatologia,Consulta Ortopedia traumatologia,62804265,Helena Noemia Zahn,(27) 99751-0594,helenazahn@gmail.com,63,2.0,Feminino,50-65
2,2023-05-31,3,SP Interior,19843,AmorSaúde Peruíbe,129.0,Ortopedia e Traumatologia,Consulta Ortopedia traumatologia,65800067,SHEILA DE OLIVEIRA HORVATE,(13) 99687-1008,sheilahorvate906@gmail.com,32,2.0,Feminino,25-35
3,2023-06-22,7,CO,19320,AmorSaúde Ceilândia Norte,188.0,Psicologia,Sessão Psicologia,63364153,Maria Eduarda Oliveira Belchior,(61) 99155-4502,raquelvuar6@gmail.com,17,2.0,Feminino,Abaixo 18
4,2023-05-29,16,SP CAV,19359,AmorSaúde SP Jabaquara,188.0,Psicologia,Retorno Psicologia,64393064,Anderson Ferreira Crispim,(11) 96286-5263,cintialainne@ymail.com,48,1.0,Masculino,35-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090487,2023-07-06,16,SP CAV,19339,AmorSaúde Guarulhos Parque São Luiz,271.0,Ginecologia,Consulta Ginecologia,13733622,VICTORIA HELEN DE LIMA,(11) 94989-0268,victoriadelima@outlook.com.br,26,2.0,Feminino,25-35
1090488,2023-06-01,9,MG,19314,AmorSaúde Betim,141.0,Psiquiatria,Sessão Psiquiatria,18390963,Valeria Barbosa Santos Galdino,(31) 98538-3103,Clientesmorsaude@outlook.com,51,2.0,Feminino,50-65
1090489,2023-07-24,3,SP Interior,19400,AmorSaúde Piracicaba,128.0,Ginecologia e Obstetrícia,Consulta Ginecologia,11450354,Vanderleia Mota Neves Papa,(19) 99890-4970,Vanderleia365@gmail.com,57,2.0,Feminino,50-65
1090490,2023-05-19,13,RJ,19563,AmorSaúde RJ Campo Grande,89.0,Clinica Médica,Consulta Clinica Médica,58815041,Beatriz Carvalho Silva Venâncio,(21) 96515-6420,BEATRZ@YAHOO.COM,25,2.0,Feminino,18-25


In [30]:
# segmentando o dataframe validado em regionais
df_spint = valid_df[valid_df['id_regional'] == 3].copy()

df_spcav = valid_df[valid_df['id_regional'] == 16].copy()

df_mg = valid_df[valid_df['id_regional'] == 9].copy()

df_ne1 = valid_df[valid_df['id_regional'] == 10].copy()

df_es = valid_df[valid_df['id_regional'] == 8].copy()

df_sul = valid_df[valid_df['id_regional'] == 6].copy()

df_rj = valid_df[valid_df['id_regional'] == 13].copy()

df_co = valid_df[valid_df['id_regional'] == 7].copy()

In [31]:
# calculando a quantidade total de entradas

pop_total = len(valid_df)
sample_size_total = pop_total * 0.15

dfs_regionais = [df_spcav, df_spint, df_mg, df_ne1, df_es, df_sul, df_rj, df_co]

# iterando sobre a lista de df
for df in dfs_regionais:
    qtd_regional = len(df)
    dcm_regional = qtd_regional / pop_total
    pct_regional = dcm_regional * 100
    qtd_necessaria = dcm_regional * sample_size_total
    pct_regional = f"{pct_regional:.2f}"
    qual_regional = df['regional'].unique()
    qtd_necessaria = f"{qtd_necessaria:.0f}"
    print(f'Regional: {qual_regional}')
    print(f'Quantidade de entradas na amostra: {qtd_regional}')
    print(f'Porcenta amostra em relação ao total: {pct_regional}%')
    print(f'Quantidade necagem dessária: {qtd_necessaria}')
    print()

Regional: ['SP CAV']
Quantidade de entradas na amostra: 223527
Porcenta amostra em relação ao total: 21.07%
Quantidade necagem dessária: 33529

Regional: ['SP Interior']
Quantidade de entradas na amostra: 155964
Porcenta amostra em relação ao total: 14.70%
Quantidade necagem dessária: 23395

Regional: ['MG']
Quantidade de entradas na amostra: 121843
Porcenta amostra em relação ao total: 11.48%
Quantidade necagem dessária: 18276

Regional: ['NE1']
Quantidade de entradas na amostra: 125604
Porcenta amostra em relação ao total: 11.84%
Quantidade necagem dessária: 18841

Regional: ['ES']
Quantidade de entradas na amostra: 49057
Porcenta amostra em relação ao total: 4.62%
Quantidade necagem dessária: 7359

Regional: ['Sul']
Quantidade de entradas na amostra: 197075
Porcenta amostra em relação ao total: 18.57%
Quantidade necagem dessária: 29561

Regional: ['RJ']
Quantidade de entradas na amostra: 86462
Porcenta amostra em relação ao total: 8.15%
Quantidade necagem dessária: 12969

Regional: 

In [32]:
# estipulando quantidades das amostras

sample_size_total = f"{sample_size_total:.0f}"
sample_size_spcav = 33529
sample_size_spint = 23395
sample_size_mg = 18276
sample_size_ne1 = 18841
sample_size_es = 7359
sample_size_sul = 29561
sample_size_rj = 12969
sample_size_co = 15220


print(sample_size_total)

159150


In [33]:
# calculando as proporções dos estratos
stratum_porportions_spcav = df_spcav.groupby(['grupo_idade', 'nomesexo']).size() / len(df_spcav)

#calculando o tamanho da amostra para cada estrato
stratum_sample_sizes = (stratum_porportions_spcav * sample_size_spcav).astype(int)

In [34]:
# criando dataframe para receber amostra estratificada
stratified_sample_spcav = pd.DataFrame()

# executando amostragem estratificada
for (grupo_idade, sexo), sample_size in stratum_sample_sizes.items():
    # filtrar o df baseado no estrato atual
    stratum = df_spcav[(df_spcav['grupo_idade'] == grupo_idade) & (df_spcav['nomesexo'] == sexo)]

    # amostrar aleatoriamente o número especificado de observações
    sampled_stratum = stratum.sample(n=sample_size, replace=False)

    # anexar o estrato amostrado ao df stratified_sample_spcav
    stratified_sample_spcav = stratified_sample_spcav.append(sampled_stratum)

# resetar o index de stratified_sampled_spcav
stratified_sample_spcav.reset_index(drop=True, inplace=True)

/tmp/ipykernel_676/483617720.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_sample_spcav = stratified_sample_spcav.append(sampled_stratum)
/tmp/ipykernel_676/483617720.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_sample_spcav = stratified_sample_spcav.append(sampled_stratum)
/tmp/ipykernel_676/483617720.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_sample_spcav = stratified_sample_spcav.append(sampled_stratum)
/tmp/ipykernel_676/483617720.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_sample_spcav = stratified_sample_spcav.append(sampled_stratum)
/tmp/ipykernel_676/4

In [35]:
# mostrando colunas no df

stratified_sample_spcav.columns

Index(['dt_atendimento', 'id_regional', 'regional', 'id_unidade',
       'nome_fantasia', 'id_especialidade', 'nome_especialidade',
       'nome_procedimento', 'id_paciente', 'nome_paciente', 'celular', 'email',
       'idade', 'sexo', 'nomesexo', 'grupo_idade'],
      dtype='object')

In [36]:
# validando se a amostra é estatisticamente semelhante à população original

# Define the observed frequencies in the stratified sample
observed_freq_sample = stratified_sample_spcav['nomesexo'].value_counts()

# Define the expected frequencies based on the proportions in the original dataframe
expected_freq_original = df_spcav['nomesexo'].value_counts() * len(stratified_sample_spcav) / len(df_spcav)

# Perform the chi-square test
chi2, p_value = stats.chisquare(observed_freq_sample, expected_freq_original)

# Compare the p-value with the significance level
alpha = 0.05
if p_value < alpha:
    print("There is a significant difference between the stratified sample and the original dataframe in terms of gender.")
else:
    print("There is no significant difference between the stratified sample and the original dataframe in terms of gender.")

There is no significant difference between the stratified sample and the original dataframe in terms of gender.


In [37]:
# exportando dfs com amostra estratificada
amostra_estratificada_spcav = 'amostra_spcav.xlsx'

stratified_sample_spcav.to_excel(amostra_estratificada_spcav, index=False)